# Americká samoobslužná kavárna - SimPy simulace

Model A: Lineární tok s FIFO frontami

**Tok zákazníka:**
1. Příchod
2. Fronta u pokladny → Objednání + Platba
3. Fronta u baru → Příprava nápoje
4. Vyzvednutí
5. Odchod nebo sezení u stolu

## 1. Instalace knihoven

Spusťte jednou (pokud nemáte nainstalováno):

In [ ]:
# Odkomentujte a spusťte při první instalaci:
# !pip install simpy matplotlib pandas numpy

## 2. Import knihoven

In [ ]:
import simpy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# Pro hezčí grafy
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 3. Parametry simulace

**Zde můžete měnit hodnoty a experimentovat!**

In [ ]:
# === PARAMETRY - ZMĚŇTE PODLE POTŘEBY ===

ARRIVAL_RATE = 15        # zákazníků/hodinu
NUM_CASHIERS = 1         # počet pokladen
NUM_BARISTAS = 2         # počet baristů
NUM_TABLES = 12          # počet stolků

# Časy obsluhy (minuty)
CASHIER_TIME_MEAN = 2.0  # průměr
CASHIER_TIME_STD = 0.5   # směrodatná odchylka

# Časy přípravy nápojů (minuty)
DRINK_TIMES = {
    'espresso': 1.5,
    'cappuccino': 3.0,
    'tea': 2.0
}

# Pravděpodobnost, že zákazník chce sednout
P_WANTS_TABLE = 0.6

# Doba konzumace u stolu (minuty)
CONSUMPTION_TIME_MEAN = 25
CONSUMPTION_TIME_STD = 10

# Délka simulace (minuty)
SIM_TIME = 300           # 5 hodin = 300 minut

print("✅ Parametry nastaveny!")
print(f"Simulace: {SIM_TIME} minut ({SIM_TIME/60:.1f} hodin)")
print(f"Příchody: {ARRIVAL_RATE} zákazníků/hodinu = každých {60/ARRIVAL_RATE:.1f} minut")

## 4. Statistiky - co budeme sledovat

In [ ]:
class Statistics:
    """Třída pro sběr statistik během simulace"""
    
    def __init__(self):
        # Časy čekání
        self.wait_cashier = []
        self.wait_drink = []
        self.time_in_system = []
        
        # Délky front v čase
        self.cashier_queue_length = []
        self.drink_queue_length = []
        self.queue_times = []
        
        # Počty
        self.total_customers = 0
        self.served_customers = 0
        self.left_unsatisfied = 0
        
    def record_queue_length(self, time, cashier_q, drink_q):
        """Zaznamenej délky front"""
        self.queue_times.append(time)
        self.cashier_queue_length.append(cashier_q)
        self.drink_queue_length.append(drink_q)
    
    def summary(self):
        """Vytiskni shrnutí statistik"""
        print("\n" + "="*50)
        print("VÝSLEDKY SIMULACE")
        print("="*50)
        print(f"\n📊 ZÁKAZNÍCI:")
        print(f"   Celkem příchodů: {self.total_customers}")
        print(f"   Obslouženo: {self.served_customers}")
        print(f"   Odešli nespokojeni: {self.left_unsatisfied}")
        
        if self.served_customers > 0:
            print(f"\n⏱️  PRŮMĚRNÉ ČASY:")
            print(f"   Čekání u pokladny: {np.mean(self.wait_cashier):.2f} min")
            print(f"   Čekání na nápoj: {np.mean(self.wait_drink):.2f} min")
            print(f"   Celkem v systému: {np.mean(self.time_in_system):.2f} min")
            
            print(f"\n📈 FRONTY (průměr):")
            print(f"   U pokladny: {np.mean(self.cashier_queue_length):.2f} zákazníků")
            print(f"   U baru: {np.mean(self.drink_queue_length):.2f} objednávek")
            
            satisfaction = 100 * (1 - self.left_unsatisfied / self.total_customers)
            print(f"\n😊 SPOKOJENOST: {satisfaction:.1f}%")
        
        print("="*50)

# Vytvoř globální statistiky
stats = Statistics()
print("✅ Statistiky připraveny!")

## 5. Definice zákazníka

**Tohle je srdce simulace - chování zákazníka**

In [ ]:
def customer(env, name, cashier, baristas, tables):
    """
    Proces zákazníka v kavárně.
    
    Args:
        env: SimPy prostředí
        name: jméno zákazníka
        cashier: zdroj pokladen
        baristas: zdroj baristů
        tables: zdroj stolků
    """
    
    arrival_time = env.now
    stats.total_customers += 1
    
    print(f"⏰ {env.now:6.2f} min | {name} přichází")
    
    # === 1. FRONTA U POKLADNY ===
    cashier_queue_start = env.now
    with cashier.request() as req:
        yield req  # Čekám na volnou pokladnu
        
        cashier_wait = env.now - cashier_queue_start
        stats.wait_cashier.append(cashier_wait)
        
        print(f"⏰ {env.now:6.2f} min | {name} u pokladny (čekal {cashier_wait:.2f} min)")
        
        # Obsluha u pokladny (normální rozdělení)
        service_time = max(0.5, random.gauss(CASHIER_TIME_MEAN, CASHIER_TIME_STD))
        yield env.timeout(service_time)
    
    # Výběr nápoje
    drink_type = random.choice(list(DRINK_TIMES.keys()))
    print(f"☕ {env.now:6.2f} min | {name} objednal {drink_type}")
    
    # === 2. FRONTA U BARU ===
    drink_queue_start = env.now
    with baristas.request() as req:
        yield req  # Čekám na volného baristu
        
        drink_wait = env.now - drink_queue_start
        stats.wait_drink.append(drink_wait)
        
        print(f"⏰ {env.now:6.2f} min | {name} - barista začíná (čekal {drink_wait:.2f} min)")
        
        # Příprava nápoje
        prep_time = DRINK_TIMES[drink_type]
        yield env.timeout(prep_time)
        
        print(f"✅ {env.now:6.2f} min | {name} - nápoj připraven")
    
    # === 3. ROZHODNUTÍ: SEDNOUT / ODNÉST ===
    wants_table = random.random() < P_WANTS_TABLE
    
    if wants_table:
        # Pokusit se sednout
        table_req = tables.request()
        
        if len(tables.users) < tables.capacity:
            # Je volný stolek
            yield table_req
            
            print(f"🪑 {env.now:6.2f} min | {name} sedí u stolu")
            
            # Konzumace
            consumption_time = max(5, random.gauss(
                CONSUMPTION_TIME_MEAN, 
                CONSUMPTION_TIME_STD
            ))
            yield env.timeout(consumption_time)
            
            tables.release(table_req)
            print(f"👋 {env.now:6.2f} min | {name} odchází ze stolu")
        else:
            # Není místo
            print(f"❌ {env.now:6.2f} min | {name} nenašel stolek, odnáší")
    else:
        print(f"🚶 {env.now:6.2f} min | {name} odnáší s sebou")
    
    # Celkový čas v systému
    total_time = env.now - arrival_time
    stats.time_in_system.append(total_time)
    stats.served_customers += 1
    
    print(f"✨ {env.now:6.2f} min | {name} odchází (celkem {total_time:.2f} min)\n")

print("✅ Chování zákazníka definováno!")

## 6. Generátor příchodů zákazníků

In [ ]:
def customer_generator(env, cashier, baristas, tables):
    """
    Generuje příchody zákazníků podle Poissonova procesu.
    """
    customer_count = 0
    
    while True:
        # Exponenciální rozdělení mezi příchody
        inter_arrival_time = random.expovariate(ARRIVAL_RATE / 60)
        yield env.timeout(inter_arrival_time)
        
        customer_count += 1
        customer_name = f"Zákazník_{customer_count}"
        
        # Vytvoř nového zákazníka
        env.process(customer(env, customer_name, cashier, baristas, tables))

print("✅ Generátor příchodů definován!")

## 7. Monitor front - sledování v čase

In [ ]:
def queue_monitor(env, cashier, baristas):
    """
    Každých 5 minut zaznamenává délky front.
    """
    while True:
        stats.record_queue_length(
            env.now,
            len(cashier.queue),
            len(baristas.queue)
        )
        yield env.timeout(5)  # Každých 5 minut

print("✅ Monitor front definován!")

## 8. SPUŠTĚNÍ SIMULACE

**🚀 Tohle je ten hlavní moment!**

In [ ]:
def run_simulation():
    """Spustí celou simulaci."""
    
    print("\n" + "="*50)
    print("🚀 SPOUŠTÍM SIMULACI")
    print("="*50 + "\n")
    
    # Vytvoř prostředí
    env = simpy.Environment()
    
    # Vytvoř zdroje (kapacity)
    cashier = simpy.Resource(env, capacity=NUM_CASHIERS)
    baristas = simpy.Resource(env, capacity=NUM_BARISTAS)
    tables = simpy.Resource(env, capacity=NUM_TABLES)
    
    # Spusť procesy
    env.process(customer_generator(env, cashier, baristas, tables))
    env.process(queue_monitor(env, cashier, baristas))
    
    # Spusť simulaci
    env.run(until=SIM_TIME)
    
    # Zobraz výsledky
    stats.summary()

# === SPUSTIT SIMULACI ===
# Resetuj statistiky
stats = Statistics()

# SPUSŤ!
run_simulation()

## 9. Vizualizace výsledků

**Krásné grafy automaticky!**

In [ ]:
# Vytvoř figure s více grafy
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Americká kavárna - Výsledky simulace', fontsize=16, fontweight='bold')

# === GRAF 1: Délky front v čase ===
ax1 = axes[0, 0]
ax1.plot(stats.queue_times, stats.cashier_queue_length, 
         label='Fronta u pokladny', color='blue', linewidth=2)
ax1.plot(stats.queue_times, stats.drink_queue_length, 
         label='Fronta u baru', color='orange', linewidth=2)
ax1.set_xlabel('Čas (minuty)')
ax1.set_ylabel('Počet čekajících')
ax1.set_title('Vývoj front v čase')
ax1.legend()
ax1.grid(True, alpha=0.3)

# === GRAF 2: Histogram časů čekání ===
ax2 = axes[0, 1]
ax2.hist(stats.wait_cashier, bins=20, alpha=0.7, label='U pokladny', color='blue')
ax2.hist(stats.wait_drink, bins=20, alpha=0.7, label='Na nápoj', color='orange')
ax2.set_xlabel('Čas čekání (minuty)')
ax2.set_ylabel('Počet zákazníků')
ax2.set_title('Rozdělení časů čekání')
ax2.legend()
ax2.grid(True, alpha=0.3)

# === GRAF 3: Celkový čas v systému ===
ax3 = axes[1, 0]
ax3.hist(stats.time_in_system, bins=30, color='green', alpha=0.7, edgecolor='black')
ax3.axvline(np.mean(stats.time_in_system), color='red', 
            linestyle='--', linewidth=2, label=f'Průměr: {np.mean(stats.time_in_system):.1f} min')
ax3.set_xlabel('Čas v systému (minuty)')
ax3.set_ylabel('Počet zákazníků')
ax3.set_title('Celková doba v kavárně')
ax3.legend()
ax3.grid(True, alpha=0.3)

# === GRAF 4: Souhrnné statistiky ===
ax4 = axes[1, 1]
metrics = ['Čekání\nu pokladny', 'Čekání\nna nápoj', 'Celkem\nv systému']
values = [
    np.mean(stats.wait_cashier),
    np.mean(stats.wait_drink),
    np.mean(stats.time_in_system)
]
colors = ['blue', 'orange', 'green']
bars = ax4.bar(metrics, values, color=colors, alpha=0.7, edgecolor='black')
ax4.set_ylabel('Čas (minuty)')
ax4.set_title('Průměrné časy')
ax4.grid(True, alpha=0.3, axis='y')

# Přidej hodnoty nad sloupce
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Grafy vytvořeny!")

## 10. Tabulka s daty (pro Excel)

**Export do pandas DataFrame - můžete uložit do CSV**

In [ ]:
# Vytvoř DataFrame se statistikami jednotlivých zákazníků
df = pd.DataFrame({
    'Čekání u pokladny (min)': stats.wait_cashier,
    'Čekání na nápoj (min)': stats.wait_drink,
    'Celkem v systému (min)': stats.time_in_system
})

# Zobraz prvních 10 zákazníků
print("\n📊 První zákazníci:")
print(df.head(10))

# Statistické shrnutí
print("\n📈 Statistické shrnutí:")
print(df.describe())

# Uložit do CSV (odkomentujte pokud chcete)
# df.to_csv('american_cafe_results.csv', index=False)
# print("\n💾 Data uložena do 'american_cafe_results.csv'")

## 11. Experimenty - Změňte parametry!

**Zkuste různé konfigurace:**

In [ ]:
# === EXPERIMENT: Porovnání různých počtů baristů ===

results = []

for num_baristas in [1, 2, 3]:
    print(f"\n🧪 Experiment: {num_baristas} baristé")
    print("="*50)
    
    # Resetuj statistiky
    stats = Statistics()
    
    # Nastav parametry
    NUM_BARISTAS = num_baristas
    
    # Vytvoř prostředí
    env = simpy.Environment()
    cashier = simpy.Resource(env, capacity=NUM_CASHIERS)
    baristas = simpy.Resource(env, capacity=NUM_BARISTAS)
    tables = simpy.Resource(env, capacity=NUM_TABLES)
    
    # Spusť (bez výpisu)
    env.process(customer_generator(env, cashier, baristas, tables))
    env.process(queue_monitor(env, cashier, baristas))
    env.run(until=SIM_TIME)
    
    # Zaznamenej výsledky
    results.append({
        'Počet baristů': num_baristas,
        'Průměrné čekání na nápoj (min)': np.mean(stats.wait_drink),
        'Průměrná fronta u baru': np.mean(stats.drink_queue_length),
        'Celkem v systému (min)': np.mean(stats.time_in_system)
    })

# Zobraz výsledky
results_df = pd.DataFrame(results)
print("\n" + "="*50)
print("📊 VÝSLEDKY EXPERIMENTŮ")
print("="*50)
print(results_df.to_string(index=False))

# Graf porovnání
fig, ax = plt.subplots(figsize=(10, 6))
x = results_df['Počet baristů']
ax.plot(x, results_df['Průměrné čekání na nápoj (min)'], 
        marker='o', linewidth=2, markersize=10, label='Čekání na nápoj')
ax.plot(x, results_df['Průměrná fronta u baru'], 
        marker='s', linewidth=2, markersize=10, label='Délka fronty')
ax.set_xlabel('Počet baristů')
ax.set_ylabel('Čas (minuty) / Počet')
ax.set_title('Vliv počtu baristů na výkonnost')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 🎯 Shrnutí

**Co jsme viděli:**

1. ✅ Americká kavárna s lineárním tokem
2. ✅ FIFO fronty u pokladny a baru
3. ✅ Statistiky a grafy
4. ✅ Experimenty s různými parametry

**Výhody SimPy oproti NetLogo:**
- 📝 Čitelný kód (skoro jako angličtina)
- 📊 Krásné grafy automaticky
- 🐍 Python - flexibilní a mocný
- 💾 Snadný export dat

**Další kroky:**
- Změňte parametry nahoře (buňka 3)
- Spusťte znovu simulaci
- Porovnejte výsledky!